In [103]:
%%bash
pip install yfinance
pip install ta

In [104]:
import yfinance as yf
apple_data = yf.download(tickers = "AAPL",  # list of tickers
            period = "10y",         # time period
            interval = "1d",       # trading interval
            ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
            prepost = False) 
# Create a new column called 'Price Change' that indicates whether the stock price went up or down by the end of the day
apple_data['Price Change'] = ['Up' if apple_data['Close'][i] > apple_data['Open'][i] else 'Down' for i in range(len(apple_data))]
print(apple_data.shape)
apple_data.head()

[*********************100%***********************]  1 of 1 completed
(2517, 7)


,Open,High,Low,Close,Adj Close,Volume,Price Change
Date,,,,,,,
2013-04-12,15.505357,15.505357,15.324643,15.350000,13.257505,238613200,Down
2013-04-15,15.250000,15.281786,14.983929,14.994643,12.950588,317520000,Down
2013-04-16,15.056071,15.236071,15.020357,15.222857,13.147696,305771200,Up
2013-04-17,15.009643,15.021429,14.218214,14.385714,12.424671,945056000,Down
2013-04-18,14.463929,14.492500,13.919286,14.001786,12.093080,666299200,Down


In [105]:
# Exponential smoothing
alpha = 0.3
import numpy as np

numeric_cols = apple_data.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    apple_data[col] = apple_data[col].ewm(alpha=0.3).mean()

In [106]:
# Generate technical indicators using the TA-Lib library
from ta.utils import dropna
import ta
import pandas as pd
apple_data = dropna(apple_data)

# List of technical indicators to calculate
indicators = ['SMA', 'EMA', 'MACD', 'RSI', 'ADX', 'BollingerBands']

# Create a new DataFrame to store the technical indicators
technical_indicators = pd.DataFrame(index=apple_data.index)

# Calculate technical indicators
technical_indicators['SMA'] = ta.trend.SMAIndicator(close=apple_data['Close'], window=20).sma_indicator()
technical_indicators['EMA'] = ta.trend.EMAIndicator(close=apple_data['Close'], window=20).ema_indicator()
macd = ta.trend.MACD(close=apple_data['Close'], window_slow=26, window_fast=12, window_sign=9).macd()
macdsignal = ta.trend.MACD(close=apple_data['Close'], window_slow=26, window_fast=12, window_sign=9).macd_signal()
technical_indicators['MACD'] = macd
technical_indicators['MACD_signal'] = macdsignal
technical_indicators['RSI'] = ta.momentum.RSIIndicator(close=apple_data['Close'], window=14).rsi()
technical_indicators['ADX'] = ta.trend.ADXIndicator(high=apple_data['High'], low=apple_data['Low'], close=apple_data['Close'], window=14).adx()
technical_indicators['BB_upper'], technical_indicators['BB_middle'], technical_indicators['BB_lower'] = ta.volatility.BollingerBands(close=apple_data['Close'], window=20).bollinger_mavg(), ta.volatility.BollingerBands(close=apple_data['Close'], window=20).bollinger_hband(), ta.volatility.BollingerBands(close=apple_data['Close'], window=20).bollinger_lband()

/opt/conda/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/opt/conda/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [107]:
technical_indicators['Price Change'] = apple_data['Price Change']
technical_indicators = technical_indicators.dropna()
technical_indicators.reset_index(inplace=True)
technical_indicators.rename(columns={'index': 'Date'}, inplace=True)
technical_indicators.sample(30)

,Date,SMA,EMA,MACD,MACD_signal,RSI,ADX,BB_upper,BB_middle,BB_lower,Price Change
756,2016-05-31,23.706858,24.211970,-0.260796,-0.527320,57.731225,44.910204,23.706858,24.731260,22.682456,Up
479,2015-04-24,31.528557,31.632767,0.155337,0.107211,72.242282,21.791098,31.528557,32.027556,31.029558,Down
753,2016-05-25,23.720684,24.060277,-0.542500,-0.708417,47.596806,54.340412,23.720684,24.831113,22.610255,Up
749,2016-05-19,24.236236,24.153707,-0.826570,-0.801780,25.012142,62.750457,24.236236,26.690021,21.782451,Down
883,2016-11-29,27.638303,27.769061,-0.171352,-0.236094,49.226049,40.132980,27.638303,28.372925,26.903680,Up
1442,2019-02-21,41.338372,41.635900,0.773643,0.614757,76.854287,53.446274,41.338372,44.525260,38.151483,Down
1462,2019-03-21,44.303677,44.522965,1.172082,0.947599,92.709111,65.281798,44.303677,46.754705,41.852649,Up
2184,2022-01-31,171.082748,168.798111,-1.913246,-0.726630,45.065853,42.682387,171.082748,182.130308,160.035189,Up
1142,2017-12-08,42.988973,42.610509,0.327387,0.539241,51.562612,34.498808,42.988973,43.624331,42.353615,Down
1944,2021-02-18,135.815149,134.523448,0.906425,1.631528,43.296014,32.133188,135.815149,139.571637,132.058662,Up
